In [1]:
from google.colab import drive
drive.mount('/content/ggdrive')

Mounted at /content/ggdrive


In [2]:
cd '/content/ggdrive/MyDrive/IOT Intrusion Detection'

/content/ggdrive/MyDrive/IOT Intrusion Detection


**Import The Libraries**

In [3]:
import pandas as pd
import numpy as np
import pickle

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

In [5]:
import tensorflow as tf

In [6]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.layers import LSTM, GRU, SimpleRNN

In [7]:
pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 KB 3.3 MB/s eta 0:00:00


In [8]:
import category_encoders as ce

**Import The Dataset**

In [9]:
df = pd.read_csv('/content/ggdrive/MyDrive/IOT Intrusion Detection/IoT Network Intrusion Dataset.csv')

Analyze the dataset

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625783 entries, 0 to 625782
Data columns (total 86 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow_ID            625783 non-null  object 
 1   Src_IP             625783 non-null  object 
 2   Src_Port           625783 non-null  int64  
 3   Dst_IP             625783 non-null  object 
 4   Dst_Port           625783 non-null  int64  
 5   Protocol           625783 non-null  int64  
 6   Timestamp          625783 non-null  object 
 7   Flow_Duration      625783 non-null  int64  
 8   Tot_Fwd_Pkts       625783 non-null  int64  
 9   Tot_Bwd_Pkts       625783 non-null  int64  
 10  TotLen_Fwd_Pkts    625783 non-null  float64
 11  TotLen_Bwd_Pkts    625783 non-null  float64
 12  Fwd_Pkt_Len_Max    625783 non-null  float64
 13  Fwd_Pkt_Len_Min    625783 non-null  float64
 14  Fwd_Pkt_Len_Mean   625783 non-null  float64
 15  Fwd_Pkt_Len_Std    625783 non-null  float64
 16  Bw

In [11]:
df.head()

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25/07/2019 03:25:53 AM,75,1,1,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26/05/2019 10:11:06 PM,5310,1,2,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,Anomaly,DoS,DoS-Synflooding
2,192.168.0.13-192.168.0.16-9020-52727-6,192.168.0.16,52727,192.168.0.13,9020,6,11/07/2019 01:24:48 AM,141,0,3,...,0.0,0.0,0.0,70.5,0.707107,71.0,70.0,Anomaly,Scan,Scan Port OS
3,192.168.0.13-192.168.0.16-9020-52964-6,192.168.0.16,52964,192.168.0.13,9020,6,04/09/2019 03:58:17 AM,151,0,2,...,0.0,0.0,0.0,151.0,0.000000,151.0,151.0,Anomaly,Mirai,Mirai-Hostbruteforceg
4,192.168.0.1-239.255.255.250-36763-1900-17,192.168.0.1,36763,239.255.255.250,1900,17,10/09/2019 01:41:18 AM,153,2,1,...,0.0,0.0,0.0,76.5,0.707107,77.0,76.0,Anomaly,Mirai,Mirai-Hostbruteforceg


**Drop the unnecessary columns**

In [12]:
df = df.drop(['Flow_ID','Src_IP','Dst_IP', 'Timestamp'], axis = 1)

**drop the infinite values**

In [13]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna(axis=0)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625415 entries, 0 to 625782
Data columns (total 82 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Src_Port           625415 non-null  int64  
 1   Dst_Port           625415 non-null  int64  
 2   Protocol           625415 non-null  int64  
 3   Flow_Duration      625415 non-null  int64  
 4   Tot_Fwd_Pkts       625415 non-null  int64  
 5   Tot_Bwd_Pkts       625415 non-null  int64  
 6   TotLen_Fwd_Pkts    625415 non-null  float64
 7   TotLen_Bwd_Pkts    625415 non-null  float64
 8   Fwd_Pkt_Len_Max    625415 non-null  float64
 9   Fwd_Pkt_Len_Min    625415 non-null  float64
 10  Fwd_Pkt_Len_Mean   625415 non-null  float64
 11  Fwd_Pkt_Len_Std    625415 non-null  float64
 12  Bwd_Pkt_Len_Max    625415 non-null  float64
 13  Bwd_Pkt_Len_Min    625415 non-null  float64
 14  Bwd_Pkt_Len_Mean   625415 non-null  float64
 15  Bwd_Pkt_Len_Std    625415 non-null  float64
 16  Fl

**Cehck the unique value of Label , Cat and Sub_cat Columns**

In [15]:
df[['Label','Cat','Sub_Cat']].nunique()

Label      2
Cat        5
Sub_Cat    9
dtype: int64

**Drop the column those have only one unique value**

In [16]:
redundant_feat = list(df.loc[:, df.nunique() == 1].columns)

In [17]:
df = df.drop(redundant_feat, axis = 1)

**Check the dataset**

In [18]:
df.shape

(625415, 72)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 625415 entries, 0 to 625782
Data columns (total 72 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Src_Port           625415 non-null  int64  
 1   Dst_Port           625415 non-null  int64  
 2   Protocol           625415 non-null  int64  
 3   Flow_Duration      625415 non-null  int64  
 4   Tot_Fwd_Pkts       625415 non-null  int64  
 5   Tot_Bwd_Pkts       625415 non-null  int64  
 6   TotLen_Fwd_Pkts    625415 non-null  float64
 7   TotLen_Bwd_Pkts    625415 non-null  float64
 8   Fwd_Pkt_Len_Max    625415 non-null  float64
 9   Fwd_Pkt_Len_Min    625415 non-null  float64
 10  Fwd_Pkt_Len_Mean   625415 non-null  float64
 11  Fwd_Pkt_Len_Std    625415 non-null  float64
 12  Bwd_Pkt_Len_Max    625415 non-null  float64
 13  Bwd_Pkt_Len_Min    625415 non-null  float64
 14  Bwd_Pkt_Len_Mean   625415 non-null  float64
 15  Bwd_Pkt_Len_Std    625415 non-null  float64
 16  Fl

In [20]:
df.corr().head()

,Src_Port,Dst_Port,Protocol,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,TotLen_Fwd_Pkts,TotLen_Bwd_Pkts,Fwd_Pkt_Len_Max,Fwd_Pkt_Len_Min,...,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
Src_Port,1.000000,-0.598266,0.463146,-0.151465,0.204532,-0.021621,-0.256976,-0.002009,-0.425783,-0.414484,...,-0.051277,0.193246,-0.058264,-0.011138,-0.048441,-0.058577,-0.220282,-0.015626,-0.171657,-0.235261
Dst_Port,-0.598266,1.000000,-0.277574,-0.063120,-0.038964,-0.095047,0.334734,0.029976,0.538194,0.508351,...,-0.015855,-0.018859,-0.017969,0.005153,-0.012181,-0.020759,-0.114246,0.003270,-0.079726,-0.125430
Protocol,0.463146,-0.277574,1.000000,-0.086463,0.305869,-0.197145,0.120479,-0.205851,0.041041,0.023629,...,-0.376431,0.336353,-0.028813,0.001335,-0.021601,-0.030947,-0.136681,-0.012140,-0.103997,-0.144367
Flow_Duration,-0.151465,-0.063120,-0.086463,1.000000,0.001260,0.215276,0.089145,0.020220,-0.048959,-0.060606,...,0.075777,-0.002849,0.318631,0.337719,0.361987,0.242934,0.802829,0.732775,0.956639,0.587744
Tot_Fwd_Pkts,0.204532,-0.038964,0.305869,0.001260,1.000000,-0.101373,0.191235,-0.089427,-0.014879,-0.033981,...,-0.111908,0.992120,0.044858,0.087303,0.072177,0.022241,-0.062680,0.019507,-0.030191,-0.075783


**Label will be our target variable**

In [21]:
df['Cat'].value_counts()

Mirai                415309
Scan                  75265
DoS                   59391
Normal                40073
MITM ARP Spoofing     35377
Name: Cat, dtype: int64

In [67]:
df['Label'].value_counts()

Anomaly    585342
Normal      40073
Name: Label, dtype: int64

These 5 classes we will be attempting to classify in our Classification Problem

**Apply Label Encoding on Label column**

In [68]:
labelencoder = LabelEncoder()
df['Label']= labelencoder.fit_transform(df['Label'])

**We have 69 independent features.**

**Label will be our target variable**

In [69]:
X = df.iloc[:,0:69]
y = df['Label']

In [70]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Label, dtype: int64

**use to_categorical method to Converts a class vector (integers) to binary class mat**rix.

In [71]:
y = tf.keras.utils.to_categorical(
    y, num_classes=None, dtype='float32'
)

In [72]:
y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [73]:
d1= pd.DataFrame(y)

In [74]:
d1.head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [75]:
y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [76]:
X[['Src_Port','Dst_Port','Protocol']].nunique()

Src_Port    11205
Dst_Port     1034
Protocol        3
dtype: int64

**Encoding Categorical Variables**

We have three Categorical Variables which are Src_Port, Dst_Port and Protocol
We need to encode this categorical data

As Categorical Data has too many categories we cannot do OneHotEncoding, as it will increase dimensionality of our feature data too much

In [77]:
X['Src_Port'].unique()

array([10000,  2179, 52727, ..., 63997, 51806,  4656])

We cannot do Label Encoding because categorical data Src_Port and Dst_Port are basically port number and thus they are categories only, there is no order between them

Thus we will be using Binary Encoding

In [78]:
encoder = ce.BinaryEncoder(cols = ['Src_Port','Dst_Port','Protocol'])

In [79]:
X = encoder.fit_transform(X)

In [80]:
X.shape

(625415, 93)

In [81]:
X.head()

,Src_Port_0,Src_Port_1,Src_Port_2,Src_Port_3,Src_Port_4,Src_Port_5,Src_Port_6,Src_Port_7,Src_Port_8,Src_Port_9,...,Init_Bwd_Win_Byts,Fwd_Act_Data_Pkts,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min
0,0,0,0,0,0,0,0,0,0,0,...,-1,1,0.0,0.0,0.0,0.0,75.0,0.000000,75.0,75.0
1,0,0,0,0,0,0,0,0,0,0,...,14600,0,0.0,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0
2,0,0,0,0,0,0,0,0,0,0,...,1869,0,0.0,0.0,0.0,0.0,70.5,0.707107,71.0,70.0
3,0,0,0,0,0,0,0,0,0,0,...,1869,0,0.0,0.0,0.0,0.0,151.0,0.000000,151.0,151.0
4,0,0,0,0,0,0,0,0,0,0,...,-1,2,0.0,0.0,0.0,0.0,76.5,0.707107,77.0,76.0


**Apply the Normalization on X data**

In [82]:
scaler = MinMaxScaler()

In [83]:
X = scaler.fit_transform(X)
# X_test = scaler.fit_transform(X_test)

**Split the data into Training and Testing**

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 100)

Check the shape 

In [85]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(500332, 93)
(500332, 2)
(125083, 93)
(125083, 2)


Covariance Matrix

In [86]:
import numpy as np

In [87]:
cm = np.cov(X.T)

In [88]:
eig_val, eig_vac = np.linalg.eig(cm)

In [89]:
import plotly.express as px
from sklearn.decomposition import PCA

In [90]:
pca = PCA()
pca.fit(X)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

px.area(
    x=range(1, exp_var_cumul.shape[0] + 1),
    y=exp_var_cumul,
    labels={"x": "# Components", "y": "Explained Variance"}
)

Create PCA model 

In [91]:
pcamodel = PCA(n_components=10)

In [92]:
x_train_new = pcamodel.fit_transform(X_train)
x_test_new = pcamodel.fit_transform(X_test)

In [93]:
x_train_new.shape

(500332, 10)

**Reshape the data according to RNN Model**

In [94]:
X_train = np.reshape(x_train_new, (x_train_new.shape[0],x_train_new.shape[1],1))
X_test = np.reshape(x_test_new, (x_test_new.shape[0],x_test_new.shape[1],1))

In [95]:
X_train.shape

(500332, 10, 1)

In [96]:
X_test.shape

(125083, 10, 1)

In [97]:
y_train.shape[1]

2

**Create RNN Model**

In [98]:
rnn = Sequential()
rnn.add(SimpleRNN(256, input_shape =(10,1), return_sequences =False))
rnn.add(Dense(units=128, input_dim=20, activation='relu'))
rnn.add(Dropout(0.2))
# rnn.add(SimpleRNN(256))
# rnn.add(Dense(units=256, activation='relu'))
# rnn.add(Dropout(0.2))
# rnn.add(Dense(units=256, activation='relu'))
# rnn.add(SimpleRNN(128))
# rnn.add(Dense(units=256, activation='relu'))
# rnn.add(Dropout(0.2))
rnn.add(Dense(units=256, activation='relu'))
rnn.add(Dropout(0.2))
rnn.add(Dense(units=128, activation='relu'))
rnn.add(Dropout(0.2))
rnn.add(Dense(units=y_train.shape[1],activation='relu'))
rnn.add(Activation('softmax'))
rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 256)               66048     
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dropout_6 (Dropout)         (None, 128)              

In [99]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.001)

In [100]:
rnn.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])

In [101]:
rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, 256)               66048     
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               33024     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dropout_6 (Dropout)         (None, 128)              

In [ ]:
# rnn.fit(X_train, y_train)

In [102]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [103]:
checkpoint = ModelCheckpoint("Resnet50.h5", monitor='accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto')
early_stop = EarlyStopping(monitor='accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
callbacks_list = [early_stop, checkpoint]

In [104]:
rnn.fit(X_train, y_train,epochs=30,batch_size=64,callbacks = callbacks_list)

Epoch 1/30
7817/7818 [============================>.] - ETA: 0s - loss: 0.0522 - accuracy: 0.9851
Epoch 1: accuracy improved from -inf to 0.98508, saving model to Resnet50.h5
7818/7818 [==============================] - 78s 10ms/step - loss: 0.0522 - accuracy: 0.9851
Epoch 2/30
7814/7818 [============================>.] - ETA: 0s - loss: 0.0265 - accuracy: 0.9927
Epoch 2: accuracy improved from 0.98508 to 0.99271, saving model to Resnet50.h5
7818/7818 [==============================] - 75s 10ms/step - loss: 0.0265 - accuracy: 0.9927
Epoch 3/30
7818/7818 [==============================] - ETA: 0s - loss: 0.0248 - accuracy: 0.9933
Epoch 3: accuracy improved from 0.99271 to 0.99327, saving model to Resnet50.h5
7818/7818 [==============================] - 76s 10ms/step - loss: 0.0248 - accuracy: 0.9933
Epoch 4/30
7815/7818 [============================>.] - ETA: 0s - loss: 0.0225 - accuracy: 0.9940
Epoch 4: accuracy improved from 0.99327 to 0.99399, saving model to Resnet50.h5
7818/7818 [=

In [105]:
rnn.evaluate(X_test, y_test)

3909/3909 [==============================] - 14s 4ms/step - loss: 0.0226 - accuracy: 0.9909


[0.022649310529232025, 0.9909340143203735]

In [106]:
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,confusion_matrix)

In [107]:
y_test_arg=np.argmax(y_test,axis=1)

In [108]:
Y_pred = np.argmax(rnn.predict(X_test),axis=1)

3909/3909 [==============================] - 10s 2ms/step


In [109]:
from sklearn.metrics import classification_report

In [110]:
print(classification_report(y_test_arg, Y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    116989
           1       1.00      0.86      0.92      8094

    accuracy                           0.99    125083
   macro avg       0.99      0.93      0.96    125083
weighted avg       0.99      0.99      0.99    125083



In [111]:
print('Confusion Matrix')
print(confusion_matrix(y_test_arg, Y_pred))

Confusion Matrix
[[116957     32]
 [  1102   6992]]
